In [21]:
import pandas as pd
import re
from collections import Counter

In [22]:
call_data = pd.read_csv("callsf0d4f5a (1).csv") 

call_reasons = pd.read_csv("reason18315ff.csv")  

call_sentiment = pd.read_csv("sentiment_statisticscc1e57a.csv")

call_data['call_id'] = call_data['call_id'].astype(str)
call_reasons['call_id'] = call_reasons['call_id'].astype(str)


merged_data = pd.merge(call_data, call_reasons, on='call_id', how='inner')


merged_data.head()



,call_id,customer_id,agent_id,call_start_datetime,agent_assigned_datetime,call_end_datetime,call_transcript,primary_call_reason
0,4667960400,2033123310,963118,7/31/2024 23:56,8/1/2024 0:03,8/1/2024 0:34,\n\nAgent: Thank you for calling United Airlin...,Voluntary Cancel
1,1122072124,8186702651,519057,8/1/2024 0:03,8/1/2024 0:06,8/1/2024 0:18,\n\nAgent: Thank you for calling United Airlin...,Booking
2,6834291559,2416856629,158319,7/31/2024 23:59,8/1/2024 0:07,8/1/2024 0:26,\n\nAgent: Thank you for calling United Airlin...,IRROPS
3,2266439882,1154544516,488324,8/1/2024 0:05,8/1/2024 0:10,8/1/2024 0:17,\n\nAgent: Thank you for calling United Airlin...,Upgrade
4,1211603231,5214456437,721730,8/1/2024 0:04,8/1/2024 0:14,8/1/2024 0:23,\n\nAgent: Thank you for calling United Airlin...,Seating


In [24]:
def clean_text(text):
    text = re.sub(r'\n', ' ', text)  # Removes new lines
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Removes non-alphabetic characters
    text = text.lower()  # Converts to lowercase
    return text

# function to clean transcripts
merged_data['cleaned_transcript'] = merged_data['call_transcript'].apply(clean_text)
merged_data[['call_transcript', 'cleaned_transcript']].head()


,call_transcript,cleaned_transcript
0,\n\nAgent: Thank you for calling United Airlin...,agent thank you for calling united airlines ...
1,\n\nAgent: Thank you for calling United Airlin...,agent thank you for calling united airlines ...
2,\n\nAgent: Thank you for calling United Airlin...,agent thank you for calling united airlines ...
3,\n\nAgent: Thank you for calling United Airlin...,agent thank you for calling united airlines ...
4,\n\nAgent: Thank you for calling United Airlin...,agent thank you for calling united airlines ...


In [25]:
# to identify self-solvable issues based on keywords
self_solvable_keywords = [
    'Voluntary Cancel', 'Booking', 'Check In', 'Seating', 'Mileage Plus',
    'Upgrade', 'Voluntary Change', 'Schedule Change', 'Digital Support'
]

def classify_issue(issue):
    #  to check if any self-solvable keyword is in the issue
    for keyword in self_solvable_keywords:
        if keyword.lower() in issue.lower():
            return 'Self-Solvable'
    return 'Non-Self-Solvable'

merged_data['self_solvable'] = merged_data['primary_call_reason'].apply(classify_issue)
merged_data[['call_id', 'primary_call_reason', 'cleaned_transcript', 'self_solvable']].head()


,call_id,primary_call_reason,cleaned_transcript,self_solvable
0,4667960400,Voluntary Cancel,agent thank you for calling united airlines ...,Non-Self-Solvable
1,1122072124,Booking,agent thank you for calling united airlines ...,Self-Solvable
2,6834291559,IRROPS,agent thank you for calling united airlines ...,Non-Self-Solvable
3,2266439882,Upgrade,agent thank you for calling united airlines ...,Self-Solvable
4,1211603231,Seating,agent thank you for calling united airlines ...,Self-Solvable


In [26]:
# Analyzing recurring problems
issue_counts = merged_data.groupby('primary_call_reason')['call_id'].count().reset_index()
issue_counts.columns = ['primary_call_reason', 'call_count']
issue_counts = issue_counts.sort_values(by='call_count', ascending=False)
issue_counts.head()


,primary_call_reason,call_count
20,IRROPS,13057
52,Voluntary Change,10291
37,Seating,6223
24,Mileage Plus,5487
30,Post-Flight,3869


In [27]:
# propose IVR improvements 
def propose_improvements(df):
   
    common_issues = df[df['self_solvable'] == 'Self-Solvable']['primary_call_reason'].value_counts()

    # a dictionary mapping issues to IVR proposals
    ivr_improvements = {
        "Booking": "Add an option for flight booking and modifications under the IVR menu.",
        "Voluntary Cancel": "Add a self-cancellation option for customers to cancel flights via IVR.",
        "Check In": "Provide an option for customers to check in for their flights using IVR.",
        "Seating": "Allow customers to choose or modify seating preferences through the IVR system.",
        "Mileage Plus": "Create an IVR option to check and manage Mileage Plus points.",
        "Upgrade": "Offer a self-service option to request flight upgrades through IVR.",
        "Voluntary Change": "Include an option for voluntary flight changes, allowing customers to reschedule their flights.",
        "Schedule Change": "Add an option for customers to handle schedule changes without agent intervention.",
        "Digital Support": "Provide assistance for digital support issues such as account login or mobile app problems through IVR."
    }

    print("Proposed IVR System Improvements Based on Common Issues:")
    for issue, count in common_issues.items():
        improvement = ivr_improvements.get(issue, "Consider adding self-service options for other recurring issues.")
        print(f"- {issue}: {improvement} (Occurred {count} times)")



In [9]:
propose_improvements(merged_data)

Proposed IVR System Improvements Based on Common Issues:
- Voluntary Change: Include an option for voluntary flight changes, allowing customers to reschedule their flights. (Occurred 10291 times)
- Seating: Allow customers to choose or modify seating preferences through the IVR system. (Occurred 6223 times)
- Mileage Plus: Create an IVR option to check and manage Mileage Plus points. (Occurred 5487 times)
- Upgrade: Offer a self-service option to request flight upgrades through IVR. (Occurred 2682 times)
- Booking: Add an option for flight booking and modifications under the IVR menu. (Occurred 2589 times)
- Voluntary Cancel: Add a self-cancellation option for customers to cancel flights via IVR. (Occurred 1304 times)
- Digital Support: Provide assistance for digital support issues such as account login or mobile app problems through IVR. (Occurred 996 times)
- Schedule Change: Add an option for customers to handle schedule changes without agent intervention. (Occurred 707 times)
- Che